In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split, Subset

from sklearn.preprocessing import MinMaxScaler

from util.env import get_device, set_device
from util.preprocess import build_loc_net, construct_data
from util.net_struct import get_feature_map, get_fc_graph_struc
from util.iostream import printsep

from datasets.TimeDataset import TimeDataset


# from models.GDN import GDN

from train import train
from test  import test
from evaluate import get_err_scores, get_best_performance_data, get_val_performance_data, get_full_err_scores

from get_data import fetchData

import sys
from datetime import datetime

import os
import argparse
from pathlib import Path

import matplotlib.pyplot as plt

import json
import random

In [2]:
# Pull data from own psql database (has attack columns in)
data = fetchData()

In [3]:
# data

In [4]:
train = data.sample(frac=0.6)
test = data.drop(train.index)

In [5]:
# train_orig = pd.read_csv(f'./data/swat/train.csv', sep=',', index_col=0)
# test_orig = pd.read_csv(f'./data/swat/test.csv', sep=',', index_col=0)

# train, test = train_orig, test_orig
if 'Normal/Attack' in train.columns:
    train = train.drop(columns=['Normal/Attack'])


In [6]:
dataset = 'swat'
feature_map = get_feature_map(dataset)
fc_struc = get_fc_graph_struc(dataset)


In [7]:
fc_edge_index = build_loc_net(fc_struc, list(train.columns), feature_map=feature_map)
fc_edge_index = torch.tensor(fc_edge_index, dtype = torch.long)

In [9]:
train_dataset_indata = construct_data(train, feature_map, labels=0)
test_dataset_indata = construct_data(test, feature_map, labels=test["Normal/Attack"].tolist())